## TEST Code for running and evaluating Steadiness / Cohesiveness and other metrics

#### Import packages & set dataset path

In [1]:
from helper import *
import sys
import importlib
import numpy as np

## Import Metrics
sys.path.append("/home/hj/vis2021_snc/metrics4mdp/metrics4mdp")
from provider import MDPMetricProvider

## Import SNC
sys.path.append("/home/hj/vis2021_snc/steadiness_cohesiveness")
import snc as sc




PATH_TO_DATASET = "../../dataset-generator/dataset/"

#### Pilot Dataset Load

In [2]:
mnist_umap_path = make_path(PATH_TO_DATASET, "mnist", "umap", 1, [0.5, 10])
emb_data, raw_data, label_data = get_data(mnist_umap_path)

raw_data = np.array(raw_data)
emb_data = np.array(emb_data)
label_data = np.array(label_data)


print(emb_data[:4])
print(len(emb_data))

[[14.0834322   4.46289539]
 [ 8.24845791  0.88252968]
 [12.69062042  1.50016272]
 [-0.71390963  4.22037649]]
10000


### Spheres TEST

In [ ]:
import time
min_dist = [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
n_neighbors = [3]

# min_dist = [0.0, 0.1]
# n_neighbors = [3, 10]


## reload SNC for the testing
importlib.reload(sc)
SNC = sc.SNC

print("TEST START")

klist=[40, 45, 50, 55, 60, 65, 70]
for k in klist:
    steadiness_result = {}
    cohesiveness_result = {}
    for md in min_dist:
        s_temp_result = {}
        c_temp_result = {}
        for nn in n_neighbors:
            mnist_umap_path = make_path(PATH_TO_DATASET, "spheres", "umap", 2, [md, nn])
            emb_data, raw_data, label_data = get_data(mnist_umap_path)

            raw_data = np.array(raw_data)
            emb_data = np.array(emb_data)
            label_data = np.array(label_data)

            parameter = {
                "k": k
            }

            start = time.time()
            snc = SNC(raw_data, emb_data, iteration=300, cluster_parameter = parameter)
            snc.fit()
            s_temp_result[nn] = snc.steadiness()
            c_temp_result[nn] = snc.cohesiveness()
            print(md,nn,"finished")
            print(s_temp_result[nn], c_temp_result[nn])
        steadiness_result[md] = s_temp_result
        cohesiveness_result[md] = c_temp_result
    data = {
        "steadiness": steadiness_result,
        "cohesivenss": cohesiveness_result
    }
    import json
    with open("./spheres_2_" + str(k) + "_result.json", "w") as outfile:
        json.dump(data, outfile)